In [20]:
import pandas as pd
import numpy as np
import pandas_profiling as pd_prof
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, RandomizedSearchCV, GridSearchCV

In [3]:
train = pd.read_csv("train_1.csv", sep="\t", index_col="Unnamed: 0")
test = pd.read_csv("test_1.csv", sep="\t", index_col="Unnamed: 0")

In [4]:
rejected_variables = ['140', '152', '160', '164', '9', '153']

In [5]:
train = train.drop(rejected_variables, axis=1, errors='ignore')
train.shape

(30500, 340)

In [6]:
x_train = train.iloc[:, 1:]
y_train = train.iloc[:, 0]
x_train.shape, y_train.shape

((30500, 339), (30500,))

In [7]:
x_train['1'].value_counts(), y_train.value_counts()

(1    29826
 0      674
 Name: 1, dtype: int64, 0    25077
 1     5423
 Name: 0, dtype: int64)

In [8]:
x_test = test.drop(rejected_variables, axis=1, errors='ignore').drop('0', axis=1)
x_test.head()

,1,2,3,4,5,6,7,8,10,11,...,336,337,338,339,340,341,342,343,344,345
0,1,0,0,1,0,0,0.136364,0,1,1,...,0.192984,0,1,0,0,0.222222,1,1,1,1
1,1,0,0,1,0,0,0.181818,0,1,1,...,0.195690,0,1,0,0,0.000000,1,1,1,0
2,1,0,0,0,0,0,0.090909,0,1,1,...,0.192984,0,1,0,0,0.222222,1,1,1,0
3,1,0,0,1,0,0,0.090909,0,1,1,...,0.195690,0,1,0,0,0.000000,1,1,1,0
4,1,0,0,1,0,0,0.090909,0,1,1,...,0.289893,0,0,1,0,0.000000,1,1,1,1


In [9]:
def lgb_score(res):
    return max(res['auc-mean']), len(res['auc-mean']) 

In [10]:
import lightgbm as lgb

In [11]:
n_rounds = 10000
kf = StratifiedKFold(n_splits=7, random_state=42)
lgb_train = lgb.Dataset(x_train, label=y_train, free_raw_data=False)

In [108]:
# test 1
lgb_param = {
    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,

    'num_leaves': 50,
    
    #regularization
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 50,
    
}




In [109]:
result = lgb.cv(lgb_param, lgb_train, n_rounds, folds=kf.split(x_train, y_train), early_stopping_rounds=170, verbose_eval=0,  )

if 'lgb_score' in vars():
    print("prev score: "+str(lgb_score))
lgb_score = max(result['auc-mean']) 
print(lgb_score, len(result['auc-mean']))

/usr/local/lib/python3.5/dist-packages/lightgbm/engine.py:390: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


prev score: <function lgb_score at 0x7f40029052f0>
0.7474241865527684 7156


In [14]:
model = lgb.train(lgb_param, lgb_train, num_boost_round=7156)

In [15]:
pred = model.predict(x_test)
print((pred))

[0.14203154 0.55405677 0.25899657 ... 0.07841536 0.10133951 0.10931236]


In [16]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, model.predict(x_train)), 0.76244713


(0.8878121290904928, 0.76244713)

In [36]:
pred_df= pd.DataFrame({'_ID_': x_test.index, '_VAL_': pred})
pred_df.to_csv('submit_1.csv', index=False)

In [110]:
# test 2
lgb_param2 = {

    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,
    'num_leaves': 50,
    
    #regularization
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 50,
    
}

    



In [111]:
%%time
#cpu


result = lgb.cv(lgb_param2, lgb_train, n_rounds, folds=kf.split(x_train, y_train), early_stopping_rounds=200, verbose_eval=0, )

if 'lgb_score2' in vars():
    print("prev score: "+str(lgb_score2))
lgb_score2 = max(result['auc-mean']) 
print(lgb_score2, len(result['auc-mean']))


0.7475685809313919 8159
CPU times: user 40min 39s, sys: 11.1 s, total: 40min 50s
Wall time: 10min 13s


In [116]:
roc_auc_score(y_train,lgb.train(lgb_param2, lgb_train, num_boost_round=8159).predict(x_train))

0.9008545253549181

In [122]:
result

{'auc-mean': [0.6850142149960851,
  0.6966472161781777,
  0.7056398883863634,
  0.708988669583085,
  0.7136942875740341,
  0.7155868611405584,
  0.715869823134659,
  0.7162192526870241,
  0.7164080531844961,
  0.7164211204381647,
  0.7163292124187345,
  0.716217223966693,
  0.7159179572227131,
  0.716049490829162,
  0.7164130180460198,
  0.7167358002026719,
  0.7169184580417374,
  0.7169731379463796,
  0.7167706868401427,
  0.7170687109054256,
  0.7173620346729873,
  0.7179605328148909,
  0.7180911260294279,
  0.7183944872856031,
  0.7184982353905551,
  0.7184217434903183,
  0.7184779896226351,
  0.7184833804988735,
  0.718650181807042,
  0.7184374784383903,
  0.7187326487733972,
  0.7187925371697651,
  0.7186959912623927,
  0.7188059013556218,
  0.7192240122069722,
  0.7195289531988435,
  0.7196211426714941,
  0.7197470402391017,
  0.7198228166462484,
  0.7197895962861656,
  0.7195779201959646,
  0.7196786260974973,
  0.7197095978527148,
  0.7197358509796521,
  0.7199189703088711,
  0

In [118]:
%%time
lgb_param3 = {
    'objective': 'binary',
    'num_threads': 5,
    "metric": 'auc',
    'learning_rate': 0.001,
    'num_leaves': 60,
    
    #regularization
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 50,
}
m = lgb.train(lgb_param3, lgb_train, num_boost_round=8159)


CPU times: user 9min 43s, sys: 2.01 s, total: 9min 45s
Wall time: 1min 58s


In [126]:
roc_auc_score(y_train,m.predict(x_train))

0.9189529110380596

In [121]:
r =lgb.cv(lgb_param3, lgb_train, n_rounds, folds=kf.split(x_train, y_train), early_stopping_rounds=300, verbose_eval=0 )


In [125]:
max(r['auc-mean']), len(r['auc-mean'])

(0.7476697993474782, 6754)

In [127]:
m2 = lgb.train(lgb_param3, lgb_train, num_boost_round=6754)
roc_auc_score(y_train,m2.predict(x_train))

0.8999449021373379

In [ ]:
pred_df= pd.DataFrame({'_ID_': x_test.index, '_VAL_': m2.pred})
pred_df.to_csv('submit_1.csv', index=False)

In [71]:
#stacking
stack_param1 = {
    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,
    'n_estimators': 10000,
    'num_leaves': 50,
    
    #regularization
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 50,
    
}
stack_num_round1 = 7156



In [72]:
def build_stack(xtrain, ytrain, xtest, param1, num_round1):
    i = list(kf.split(xtrain, ytrain))
    
    new_train_feature = np.zeros(xtrain.shape[0])
    new_test_feature = np.zeros(xtest.shape[0])
    
    lgb_ds = lgb.Dataset(xtrain, label=ytrain, free_raw_data=False)

    for (i1, (sub_train, sub_test)) in enumerate(i):
        model = lgb.train(param1, lgb_ds.subset(sub_train), num_boost_round=num_round1)
        pred = model.predict(xtrain.loc[sub_test])
        new_train_feature[sub_test] =  pred
    
    ## adding new feature to test
    model2 = lgb.train(param1, lgb_ds, num_boost_round=num_round1)
    pred2 = model2.predict(xtest)
    new_test_feature[:]= pred2
    return (pd.concat([xtrain, pd.DataFrame(new_train_feature)], axis=1), pd.concat([xtest, pd.DataFrame(new_test_feature)], axis=1))


In [73]:
%%time
new_x_train, new_x_test = build_stack(x_train, y_train, x_test, lgb_param, 7156)

CPU times: user 40min 10s, sys: 11.1 s, total: 40min 21s
Wall time: 10min 8s


In [28]:
param = {
    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,

    'num_leaves': 40,
    'max_bin' : 120,
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 40,
    
}
result = lgb.cv(param, lgb.Dataset(new_x_train, y_train, free_raw_data=False), n_rounds, folds=kf.split(new_x_train, y_train), early_stopping_rounds=300, verbose_eval=-2, )


In [29]:
max(result['auc-mean']), len(result['auc-mean']) 

(0.74422504821609969, 8)

In [76]:
#save
param = {
    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,

    'num_leaves': 40,
    'max_bin' : 120,
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 40,
    
}
model2 = lgb.train(param, lgb.Dataset(new_x_train, y_train, free_raw_data=False), num_boost_round=8)
pred = model2.predict(new_x_test)
print((pred))

[0.4971461  0.50205562 0.49890478 ... 0.49662798 0.49677052 0.49677052]


In [31]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_train, model.predict(new_x_train))

0.75592546522265536

In [32]:
pred_df= pd.DataFrame({'_ID_': new_x_test.index, '_VAL_': model.predict(new_x_test)})
pred_df.to_csv('submit_stacking.csv', index=False)

In [21]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
rfc = RandomForestClassifier(n_estimators=10, random_state=42, max_depth=4, n_jobs=4)


In [31]:
%%time
params = {
    'max_depth': [10,20,50],
    'max_features': [0.6, 0.8, 1.0],
    
    
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy']
    
}
rfcv = GridSearchCV(rfc, params, n_jobs=4, cv=kf, verbose=1, scoring='roc_auc')
rfcv.fit(x_train, y_train)
print(rfcv.best_score_)

Fitting 7 folds for each of 18 candidates, totalling 126 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  1.4min
[Parallel(n_jobs=4)]: Done 126 out of 126 | elapsed:  5.6min finished


0.712648246483727
CPU times: user 34 s, sys: 224 ms, total: 34.2 s
Wall time: 5min 41s


In [75]:
params = {
    'max_depth': [10,20,50],
    'max_features': [0.6, 0.8, 1.0],
    
    
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy']
    
}
etc = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=4)
etcv = GridSearchCV(etc, params, n_jobs=4, cv=kf, verbose=1, scoring='roc_auc')
etcv.fit(x_train, y_train)
print(etcv.best_score_)

Fitting 7 folds for each of 18 candidates, totalling 126 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 14.2min
[Parallel(n_jobs=4)]: Done 126 out of 126 | elapsed: 47.1min finished


0.7263825945795191


In [54]:
rf.predict_proba(x_train)[:,1]

array([0.69952122, 0.6919447 , 0.5936572 , ..., 0.482439  , 0.69970172,
       0.82913999])

In [77]:
rf = rfcv.best_estimator_
lg = model
et = etcv.best_estimator_

roc_auc_score(y_train, rf.predict_proba(x_train)[:,1]), roc_auc_score(y_train, lg.predict(x_train)), roc_auc_score(y_train, et.predict_proba(x_train)[:,1])

(0.8520414949725452, 0.8878121290904928, 0.8210388455704688)

In [78]:
rf_pred = rf.predict_proba(x_train)[:,1]
lg_pred = lg.predict(x_train)
et_pred = et.predict_proba(x_train)[:,1]

In [82]:
roc_auc_score(y_train, rf_pred*0.05+lg_pred*.9 + et_pred*0.05)

0.8865032267093472

In [67]:
pred_df= pd.DataFrame({'_ID_': x_test.index, '_VAL_': rf_pred*.2+lg_pred*.8})
pred_df.to_csv('submit_ans.csv', index=False)

In [84]:
%%time
## new feauture

params = {
    'max_depth': [10,20,50],
    'max_features': [0.6, 0.8, 1.0],
    
    
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy']
    
}
rfcv = RandomizedSearchCV(rfc, params, n_jobs=4, cv=kf, verbose=1, scoring='roc_auc')
rfcv.fit(new_x_train, y_train)
print(rfcv.best_score_)


Fitting 7 folds for each of 10 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.3min
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed:  3.8min finished


0.7298555179231226
CPU times: user 23.6 s, sys: 164 ms, total: 23.7 s
Wall time: 3min 51s


In [85]:
params = {
    'max_depth': [10,20,50],
    'max_features': [0.6, 0.8, 1.0],
    
    
    'class_weight': ['balanced'],
    'criterion': ['gini', 'entropy']
    
}
etc = ExtraTreesClassifier(n_estimators=100, random_state=42, n_jobs=4)
etcv = RandomizedSearchCV(etc, params, n_jobs=4, cv=kf, verbose=1, scoring='roc_auc')
etcv.fit(new_x_train, y_train)
print(etcv.best_score_)

Fitting 7 folds for each of 10 candidates, totalling 70 fits


[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed: 24.9min
[Parallel(n_jobs=4)]: Done  70 out of  70 | elapsed: 36.4min finished


0.744650412311694


In [98]:
params = {
    'objective': 'binary',
    'num_threads': 4,
    "metric": 'auc',
    'learning_rate': 0.001,

    'num_leaves': 40,
    'max_bin' : 120,
    #regularization
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'subsample_freq': 1,
    'min_data_in_leaf': 40,
    
}
model3 = lgb.train(params, lgb.Dataset(new_x_train, y_train, free_raw_data=False), num_boost_round=8)
lg_pred2 = model3.predict(new_x_train)
roc_auc_score(y_train, lg_pred2)

0.7560056497497941

In [93]:
rf_pred2 = rfcv.best_estimator_.predict_proba(new_x_train)[:,1]
et_pred2 = etcv.best_estimator_.predict_proba(new_x_train)[:,1]

In [94]:
rf_pred2.shape, et_pred2.shape

((30500,), (30500,))

In [107]:
roc_auc_score(y_train,(rf_pred+lg_pred2 + et_pred2)/3), 0.8878121290904928

(0.8493799598803086, 0.8878121290904928)